In [ ]:
import pandas as pd
import os
from rich.console import Console
from rich.table import Table
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import KFold

In [43]:
os.chdir(r'C:\SML_Projects\SML_hotelBooking_cancelling_prediction')
os.makedirs('results', exist_ok=True)

In [44]:
x_train = pd.read_csv('data/preprocessed/preprocessed_x_train.csv')
x_test = pd.read_csv('data/preprocessed/preprocessed_x_test.csv')

y_train = pd.read_csv('data/split/y_train.csv')['is_canceled']
y_test = pd.read_csv('data/split/y_test.csv')['is_canceled']

In [45]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [46]:
from collections import Counter
from imblearn.over_sampling import SMOTE

print("Original train class distribution:", Counter(y_train))
smote = SMOTE(random_state=42, k_neighbors=1)

x_train, y_train = smote.fit_resample(x_train, y_train)
print("After SMOTE train class distribution:", Counter(y_train))

Original train class distribution: Counter({0: 60259, 1: 35253})
After SMOTE train class distribution: Counter({1: 60259, 0: 60259})


In [47]:
import optuna
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.linear_model import (
    LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    HistGradientBoostingClassifier, AdaBoostClassifier,
    BaggingClassifier, VotingClassifier, StackingClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [48]:
base_estimators = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss')),
    ('lgbm', LGBMClassifier(random_state=42))
]

In [49]:
def get_model(trial, name):

    # ===== Linear =====
    if name == 'Logistic Regression':
        return LogisticRegression(
            C=trial.suggest_float('C', 1e-2, 10.0, log=True),
            solver=trial.suggest_categorical('solver', ['lbfgs', 'liblinear']),
            max_iter=1000
        )

    if name == 'RidgeClassifier':
        return RidgeClassifier(
            alpha=trial.suggest_float('alpha', 1e-2, 10.0, log=True)
        )

    if name == 'SGDC':
        return SGDClassifier(
            alpha=trial.suggest_float('alpha', 1e-4, 1.0, log=True),
            loss=trial.suggest_categorical('loss', ['log_loss', 'hinge']),
            penalty=trial.suggest_categorical('penalty', ['l2', 'l1', 'elasticnet']),
            l1_ratio=trial.suggest_float('l1_ratio', 0.1, 0.9),
            random_state=42
        )

    if name == 'Passive Aggressive':
        return PassiveAggressiveClassifier(
            C=trial.suggest_float('C', 1e-2, 10.0, log=True),
            loss=trial.suggest_categorical('loss', ['hinge', 'squared_hinge']),
            random_state=42
        )

    # ===== Tree =====
    if name == 'DecisionTree':
        return DecisionTreeClassifier(
            max_depth=trial.suggest_int('max_depth', 2, 30),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
            random_state=42
        )

    if name == 'RandomForest':
        return RandomForestClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 600),
            max_depth=trial.suggest_int('max_depth', 3, 30),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 4),
            max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2']),
            n_jobs=-1,
            random_state=42
        )

    if name == 'ExtraTrees':
        return ExtraTreesClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 600),
            max_depth=trial.suggest_int('max_depth', 3, 30),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 5),
            n_jobs=-1,
            random_state=42
        )

    if name == 'Bagging':
        return BaggingClassifier(
            n_estimators=trial.suggest_int('n_estimators', 50, 400),
            max_samples=trial.suggest_float('max_samples', 0.5, 1.0),
            max_features=trial.suggest_float('max_features', 0.5, 1.0),
            random_state=42
        )

    if name == 'Bagged DT':
        return BaggingClassifier(
            estimator=DecisionTreeClassifier(),
            n_estimators=trial.suggest_int('n_estimators', 50, 400),
            random_state=42
        )

    # ===== Boosting =====
    if name == 'GradientBoosting':
        return GradientBoostingClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            max_depth=trial.suggest_int('max_depth', 2, 6),
            random_state=42
        )

    if name == 'HistGradientBoosting':
        return HistGradientBoostingClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            max_depth=trial.suggest_int('max_depth', 2, 10),
            random_state=42
        )

    if name == 'AdaBoost':
        return AdaBoostClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
            random_state=42
        )

    if name == 'XGBoost':
        return XGBClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 3, 10),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
            eval_metric='logloss',
            random_state=42
        )

    if name == 'LGBMClassifier':
        return LGBMClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            num_leaves=trial.suggest_int('num_leaves', 31, 255),
            random_state=42
        )

    if name == 'CatBoost':
        return CatBoostClassifier(
            iterations=trial.suggest_int('iterations', 300, 800),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            depth=trial.suggest_int('depth', 4, 10),
            verbose=0,
            random_state=42
        )

    # ===== Other =====
    if name == 'SVC':
        return SVC(
            C=trial.suggest_float('C', 1e-2, 10.0, log=True),
            kernel=trial.suggest_categorical('kernel', ['rbf', 'linear']),
            gamma=trial.suggest_categorical('gamma', ['scale', 'auto']),
            probability=True
        )

    if name == 'KNN':
        return KNeighborsClassifier(
            n_neighbors=trial.suggest_int('n_neighbors', 3, 30),
            weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
            p=trial.suggest_int('p', 1, 2)
        )

    if name == 'MLPC':
        return MLPClassifier(
            hidden_layer_sizes=trial.suggest_categorical(
                'hidden_layer_sizes', [(50,), (100,), (100, 50)]
            ),
            alpha=trial.suggest_float('alpha', 1e-4, 1.0, log=True),
            learning_rate_init=trial.suggest_float('lr', 1e-4, 1e-2, log=True),
            max_iter=500,
            random_state=42
        )

    # ===== Ensembles =====
    if name == 'Hard Voting':
        return VotingClassifier(estimators=base_estimators, voting='hard')

    if name == 'Soft Voting':
        return VotingClassifier(estimators=base_estimators, voting='soft')

    if name == 'Stacking':
        return StackingClassifier(
            estimators=base_estimators,
            final_estimator=LogisticRegression(max_iter=1000)
        )

    if name == 'DummyClassifier':
        return DummyClassifier(
            strategy=trial.suggest_categorical(
                'strategy', ['most_frequent', 'stratified', 'uniform']
            )
        )

In [50]:
def objective(trial, name):
    model = get_model(trial, name)

    scores = cross_val_score(
        model,
        x_train,
        y_train,
        cv=kf,
        scoring='recall',
        n_jobs=-1
    )

    return scores.mean()


In [51]:
models = [
    'SGDC', 'Bagging', 'Stacking', 'LGBMClassifier', 'XGBoost', 'CatBoost',
    'HistGradientBoosting', 'MLPC', 'Hard Voting', 'RidgeClassifier',
    'Soft Voting', 'RandomForest', 'DecisionTree', 'GradientBoosting',
    'Bagged DT', 'SVC', 'KNN', 'ExtraTrees', 'Passive Aggressive',
    'Logistic Regression', 'AdaBoost', 'DummyClassifier'
]

results = []

for name in models:
    print("\n" + "=" * 60)
    print(f"🔥 NOW TUNING: {name}")
    print("=" * 60)

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, name), n_trials=30)

    best_model = get_model(study.best_trial, name)
    best_model.fit(x_train, y_train)

    y_pred = best_model.predict(x_test)

    # ===== Test metrics =====
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # ===== K-Fold metrics =====
    kf_scores = cross_val_score(
        best_model,
        x_train,
        y_train,
        cv=kf,
        scoring='f1_macro',
        n_jobs=-1
    )

    kf_mean = kf_scores.mean()
    kf_std = kf_scores.std()

    # ===== PRINT RESULTS =====
    print(f"\n📊 RESULTS FOR: {name}")
    print("-" * 40)
    print(f"ACCURACY     : {acc:.4f}")
    print(f"PRECISION    : {prec:.4f}")
    print(f"RECALL       : {rec:.4f}")
    print(f"F1 SCORE     : {f1:.4f}")
    print(f"K-FOLD MEAN  : {kf_mean:.4f}")
    print(f"K-FOLD STD   : {kf_std:.4f}")
    print("-" * 40)

    results.append([
        name,
        acc,
        prec,
        rec,
        f1,
        kf_mean,
        kf_std
    ])


[I 2025-12-30 19:27:48,565] A new study created in memory with name: no-name-3c009223-1b3c-42d6-bf0e-630491414b6c



🔥 NOW TUNING: SGDC


[I 2025-12-30 19:27:50,142] Trial 0 finished with value: 0.7498851901280128 and parameters: {'alpha': 0.024404106082471344, 'loss': 'hinge', 'penalty': 'l2', 'l1_ratio': 0.36824954315095737}. Best is trial 0 with value: 0.7498851901280128.
[I 2025-12-30 19:27:51,405] Trial 1 finished with value: 0.7664589496663329 and parameters: {'alpha': 0.0007265327329036694, 'loss': 'log_loss', 'penalty': 'l1', 'l1_ratio': 0.10945511773209704}. Best is trial 1 with value: 0.7664589496663329.
[I 2025-12-30 19:27:52,631] Trial 2 finished with value: 0.7743259185486023 and parameters: {'alpha': 0.02592430499238967, 'loss': 'log_loss', 'penalty': 'l2', 'l1_ratio': 0.4785803142229095}. Best is trial 2 with value: 0.7743259185486023.
[I 2025-12-30 19:27:53,843] Trial 3 finished with value: 0.7336103385979067 and parameters: {'alpha': 0.006729903920130239, 'loss': 'hinge', 'penalty': 'l2', 'l1_ratio': 0.6387908465083003}. Best is trial 2 with value: 0.7743259185486023.
[I 2025-12-30 19:27:55,098] Trial 4 


📊 RESULTS FOR: SGDC
----------------------------------------
ACCURACY     : 0.3757
PRECISION    : 0.3757
RECALL       : 1.0000
F1 SCORE     : 0.5462
K-FOLD MEAN  : 0.3333
K-FOLD STD   : 0.0003
----------------------------------------

🔥 NOW TUNING: Bagging


[I 2025-12-30 19:31:04,391] Trial 0 finished with value: 0.890491113861947 and parameters: {'n_estimators': 391, 'max_samples': 0.6200956159564395, 'max_features': 0.9353368380701383}. Best is trial 0 with value: 0.890491113861947.
[I 2025-12-30 19:33:22,679] Trial 1 finished with value: 0.8926978913720337 and parameters: {'n_estimators': 321, 'max_samples': 0.8151229664071127, 'max_features': 0.838402240397144}. Best is trial 1 with value: 0.8926978913720337.
[I 2025-12-30 19:34:13,380] Trial 2 finished with value: 0.8882505000599838 and parameters: {'n_estimators': 159, 'max_samples': 0.5232294738940722, 'max_features': 0.8425505563568015}. Best is trial 1 with value: 0.8926978913720337.
[I 2025-12-30 19:35:25,808] Trial 3 finished with value: 0.8928467773364913 and parameters: {'n_estimators': 147, 'max_samples': 0.8767186429361205, 'max_features': 0.9174245110995658}. Best is trial 3 with value: 0.8928467773364913.
[I 2025-12-30 19:36:38,459] Trial 4 finished with value: 0.89123727

...


📊 RESULTS FOR: Logistic Regression
----------------------------------------
ACCURACY     : 0.7907
PRECISION    : 0.7003
RECALL       : 0.7743
F1 SCORE     : 0.7355
K-FOLD MEAN  : 0.7862
K-FOLD STD   : 0.0008
----------------------------------------

🔥 NOW TUNING: AdaBoost


[I 2025-12-31 16:34:59,264] Trial 0 finished with value: 0.7938649771201436 and parameters: {'n_estimators': 258, 'learning_rate': 0.8604835707640708}. Best is trial 0 with value: 0.7938649771201436.
[I 2025-12-31 16:35:14,151] Trial 1 finished with value: 0.3272383282895162 and parameters: {'n_estimators': 206, 'learning_rate': 0.021331842321653378}. Best is trial 0 with value: 0.7938649771201436.
[I 2025-12-31 16:35:47,090] Trial 2 finished with value: 0.7371398704603521 and parameters: {'n_estimators': 461, 'learning_rate': 0.04022088951731519}. Best is trial 0 with value: 0.7938649771201436.
[I 2025-12-31 16:36:10,119] Trial 3 finished with value: 0.3272383282895162 and parameters: {'n_estimators': 315, 'learning_rate': 0.01463619345761134}. Best is trial 0 with value: 0.7938649771201436.
[I 2025-12-31 16:36:35,052] Trial 4 finished with value: 0.7858318446902016 and parameters: {'n_estimators': 345, 'learning_rate': 0.37857203901380787}. Best is trial 0 with value: 0.7938649771201


📊 RESULTS FOR: AdaBoost
----------------------------------------
ACCURACY     : 0.8282
PRECISION    : 0.7582
RECALL       : 0.7968
F1 SCORE     : 0.7770
K-FOLD MEAN  : 0.8275
K-FOLD STD   : 0.0017
----------------------------------------

🔥 NOW TUNING: DummyClassifier


[I 2025-12-31 16:48:39,488] Trial 0 finished with value: 0.5004871478847565 and parameters: {'strategy': 'uniform'}. Best is trial 0 with value: 0.5004871478847565.
[I 2025-12-31 16:48:39,808] Trial 1 finished with value: 0.4989967052603787 and parameters: {'strategy': 'uniform'}. Best is trial 0 with value: 0.5004871478847565.
[I 2025-12-31 16:48:40,127] Trial 2 finished with value: 0.4989146193285861 and parameters: {'strategy': 'stratified'}. Best is trial 0 with value: 0.5004871478847565.
[I 2025-12-31 16:48:40,445] Trial 3 finished with value: 0.6666666666666666 and parameters: {'strategy': 'most_frequent'}. Best is trial 3 with value: 0.6666666666666666.
[I 2025-12-31 16:48:40,766] Trial 4 finished with value: 0.6666666666666666 and parameters: {'strategy': 'most_frequent'}. Best is trial 3 with value: 0.6666666666666666.
[I 2025-12-31 16:48:41,095] Trial 5 finished with value: 0.6666666666666666 and parameters: {'strategy': 'most_frequent'}. Best is trial 3 with value: 0.6666666


📊 RESULTS FOR: DummyClassifier
----------------------------------------
ACCURACY     : 0.6243
PRECISION    : 0.0000
RECALL       : 0.0000
F1 SCORE     : 0.0000
K-FOLD MEAN  : 0.3331
K-FOLD STD   : 0.0001
----------------------------------------


In [52]:
console = Console()

result_sorted = sorted(results, key=lambda i: i[3], reverse=True)

best_model = max(results, key=lambda x: x[3])
worst_model = min(results, key=lambda x: x[3])

table = Table(title="Optuna", show_lines=True)
table.add_column("Algorithm")
table.add_column("Accuracy")
table.add_column("Precision")
table.add_column("Recall")
table.add_column("F1-score")
table.add_column("K-Fold mean")
table.add_column("K-Fold std")

for row in result_sorted:
    algo, acc, presicion, recall, f1, kmean, kstd = row

    if row == best_model:
        table.add_row(
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{acc:.2f}[/bold green]",
            f"[bold green]{presicion:.2f}[/bold green]",
            f"[bold green]{recall:.2f}[/bold green]",
            f"[bold green]{f1:.2f}[/bold green]",
            f"[bold green]{kmean:.2f}[/bold green]",
            f"[bold green]{kstd:.2f}[/bold green]",
        )
    elif row == worst_model:
        table.add_row(
            f"[bold red]{algo}[/bold red]",
            f"[bold red]{acc:.2f}[/bold red]",
            f"[bold red]{presicion:.2f}[/bold red]",
            f"[bold red]{recall:.2f}[/bold red]",
            f"[bold red]{f1:.2f}[/bold red]",
            f"[bold red]{kmean:.2f}[/bold red]",
            f"[bold red]{kstd:.2f}[/bold red]",
        )
    else:
        table.add_row(algo, f"{acc:.2f}", f"{presicion:.2f}", f"{recall:.2f}", f"{f1:.2f}", f"{kmean:.2f}", f"{kstd:.2f}")

console.print(table)

                                            Optuna                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Algorithm            ┃ Accuracy ┃ Precision ┃ Recall ┃ F1-score ┃ K-Fold mean ┃ K-Fold std ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ SGDC                 │ 0.38     │ 0.38      │ 1.00   │ 0.55     │ 0.33        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Stacking             │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagged DT            │ 0.88     │ 0.85      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagging              │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ LGBMClassifier       │ 0.88     │ 0.86      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ GradientBoosting     │ 0.88     │ 0.85      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ HistGradientBoosting │ 0.87     │ 0.84      │ 0.82   │ 0.83     │ 0.89        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Soft Voting          │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ CatBoost             │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ XGBoost              │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Hard Voting          │ 0.88     │ 0.85      │ 0.82   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ RandomForest         │ 0.88     │ 0.87      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ ExtraTrees           │ 0.87     │ 0.84      │ 0.81   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ SVC                  │ 0.84     │ 0.77      │ 0.81   │ 0.79     │ 0.84        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ DecisionTree         │ 0.85     │ 0.81      │ 0.80   │ 0.80     │ 0.86        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ AdaBoost             │ 0.83     │ 0.76      │ 0.80   │ 0.78     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Passive Aggressive   │ 0.76     │ 0.65      │ 0.79   │ 0.71     │ 0.73        │ 0.02       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ KNN                  │ 0.78     │ 0.68      │ 0.79   │ 0.73     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ MLPC                 │ 0.84     │ 0.78      │ 0.78   │ 0.78     │ 0.85        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Logistic

In [53]:
temp_console = Console(record=True)
temp_console.print(table)
text = temp_console.export_text()
with open('results/Tuning.txt', 'a', encoding='utf-8') as f:
    f.write(text)

                                            Optuna                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Algorithm            ┃ Accuracy ┃ Precision ┃ Recall ┃ F1-score ┃ K-Fold mean ┃ K-Fold std ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ SGDC                 │ 0.38     │ 0.38      │ 1.00   │ 0.55     │ 0.33        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Stacking             │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagged DT            │ 0.88     │ 0.85      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagging              │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ LGBMClassifier       │ 0.88     │ 0.86      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ GradientBoosting     │ 0.88     │ 0.85      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ HistGradientBoosting │ 0.87     │ 0.84      │ 0.82   │ 0.83     │ 0.89        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Soft Voting          │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ CatBoost             │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ XGBoost              │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Hard Voting          │ 0.88     │ 0.85      │ 0.82   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ RandomForest         │ 0.88     │ 0.87      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ ExtraTrees           │ 0.87     │ 0.84      │ 0.81   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ SVC                  │ 0.84     │ 0.77      │ 0.81   │ 0.79     │ 0.84        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ DecisionTree         │ 0.85     │ 0.81      │ 0.80   │ 0.80     │ 0.86        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ AdaBoost             │ 0.83     │ 0.76      │ 0.80   │ 0.78     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Passive Aggressive   │ 0.76     │ 0.65      │ 0.79   │ 0.71     │ 0.73        │ 0.02       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ KNN                  │ 0.78     │ 0.68      │ 0.79   │ 0.73     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ MLPC                 │ 0.84     │ 0.78      │ 0.78   │ 0.78     │ 0.85        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Logistic

# Another Tunings

<a href="../Tuning/manual_search.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Manual Search
    </button>
</a>

<a href="../Tuning/optuna.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        optuna
    </button>
</a>

<a href="../Tuning/random_search.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Random Search
    </button>
</a>